In [1]:
import pandas as pd
from pathlib import Path

In [2]:
DATA_DIR = Path("../datasets")

df_ventas = pd.read_csv(DATA_DIR / 'ventas_completo.csv')
df_clasificados = pd.read_csv(DATA_DIR / 'productos_clasificados.csv')


df_ventas.head(3)

,fecha,product_name,category_off,perecedero,ventas,precio,en_promocion,en_temporada,temp_inicio_mes,temp_fin_mes
0,2022-12-01,Néctar de durazno 1 L,juice-box,0,10,33.72,0,0,7.0,9.0
1,2022-12-02,Néctar de durazno 1 L,juice-box,0,5,33.66,0,0,7.0,9.0
2,2022-12-03,Néctar de durazno 1 L,juice-box,0,7,31.48,0,0,7.0,9.0


In [3]:
df_clasificados = df_clasificados[['product_name', 'perecedero']]
df_clasificados.head(3)

,product_name,perecedero
0,Néctar de durazno 1 L,0
1,Sardinas en salsa 155 g,0
2,Queso gouda 300 g,1


In [4]:
df_ventas_cp = df_ventas[['fecha', 'product_name', 'category_off', 'ventas', 'precio']]
df_ventas_cp.head(3)

,fecha,product_name,category_off,ventas,precio
0,2022-12-01,Néctar de durazno 1 L,juice-box,10,33.72
1,2022-12-02,Néctar de durazno 1 L,juice-box,5,33.66
2,2022-12-03,Néctar de durazno 1 L,juice-box,7,31.48


In [11]:
df_ventas_perecederos = pd.merge(df_ventas_cp, df_clasificados, how="left", on= "product_name")
df_ventas_perecederos.loc[df_ventas_perecederos['category_off'] == 'fruits-vegetables', 'perecedero'] = 1
df_ventas_perecederos[df_ventas_perecederos['perecedero'].isna()]

,fecha,product_name,category_off,ventas,precio,perecedero
